In [34]:
import os
import polars as pl
import utils

os.chdir(os.getcwd())

In [35]:
ridership_df = pl.read_csv('data/CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals_20250203.csv')

In [36]:
ridership_df.head(10)

station_id,stationname,date,daytype,rides
i64,str,str,str,i64
41280,"""Jefferson Park""","""12/22/2017""","""W""",6104
41000,"""Cermak-Chinatown""","""12/18/2017""","""W""",3636
40280,"""Central-Lake""","""12/02/2017""","""A""",1270
40140,"""Dempster-Skokie""","""12/19/2017""","""W""",1759
40690,"""Dempster""","""12/03/2017""","""U""",499
41660,"""Lake/State""","""12/30/2017""","""A""",8615
40180,"""Oak Park-Forest Park""","""12/17/2017""","""U""",442
40250,"""Kedzie-Homan-Forest Park""","""12/02/2017""","""A""",1353
40120,"""35th/Archer""","""12/07/2017""","""W""",3353


In [37]:
# Generate int columns for year, month, and day from the date column
ridership_df = ridership_df.with_columns(
    pl.col('date').str.to_date('%m/%d/%Y')
)

ridership_df = ridership_df.with_columns(
    year=pl.col('date').dt.year(),
    month=pl.col('date').dt.month(),
    day=pl.col('date').dt.day()
)

In [38]:
# Filter to a restricted date range to limit the size of the data in memory
min_year = 2018
max_year = 2024

ridership_df = ridership_df.filter(
    pl.col('year').is_between(min_year, max_year)
)

ridership_df = ridership_df.sort(['station_id', 'year', 'month', 'day'])

In [39]:
ridership_df.head(10)

station_id,stationname,date,daytype,rides,year,month,day
i64,str,date,str,i64,i32,i8,i8
40010,"""Austin-Forest Park""",2018-01-01,"""U""",343,2018,1,1
40010,"""Austin-Forest Park""",2018-01-02,"""W""",1187,2018,1,2
40010,"""Austin-Forest Park""",2018-01-03,"""W""",1429,2018,1,3
40010,"""Austin-Forest Park""",2018-01-04,"""W""",1447,2018,1,4
40010,"""Austin-Forest Park""",2018-01-05,"""W""",1391,2018,1,5
40010,"""Austin-Forest Park""",2018-01-06,"""A""",532,2018,1,6
40010,"""Austin-Forest Park""",2018-01-07,"""U""",450,2018,1,7
40010,"""Austin-Forest Park""",2018-01-08,"""W""",1773,2018,1,8
40010,"""Austin-Forest Park""",2018-01-09,"""W""",1808,2018,1,9


In [40]:
# Check that station ID, year, month, and day uniquely identify rows in the dataset 
primary_key_columns = ["station_id", "year", "month", "day"]

assert ridership_df.unique(subset=primary_key_columns).height == ridership_df.height, "Station ID, year, month, and day are not the primary keys."

In [41]:
# Load the dataset containing stop information to left join
system_info_df = pl.read_csv('data/CTA_-_System_Information_-_List_of__L__Stops_20250203.csv')

system_info_df.head(10)

STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
i64,str,str,str,str,i64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str
30082,"""E""","""Cicero (Loop-bound)""","""Cicero""","""Cicero (Pink Line)""",40420,true,false,false,false,false,false,false,false,true,false,"""(41.85182, -87.745336)"""
30151,"""E""","""Central Park (Loop-bound)""","""Central Park""","""Central Park (Pink Line)""",40780,true,false,false,false,false,false,false,false,true,false,"""(41.853839, -87.714842)"""
30184,"""W""","""Halsted/63rd (Ashland-bound)""","""Halsted""","""Halsted (Green Line)""",40940,true,false,false,true,false,false,false,false,false,false,"""(41.778943, -87.644244)"""
30044,"""N""","""Cumberland (O'Hare-bound)""","""Cumberland""","""Cumberland (Blue Line)""",40230,true,false,true,false,false,false,false,false,false,false,"""(41.984246, -87.838028)"""
30092,"""E""","""Racine (O'Hare-bound)""","""Racine""","""Racine (Blue Line)""",40470,false,false,true,false,false,false,false,false,false,false,"""(41.87592, -87.659458)"""
30253,"""N""","""Paulina (Kimball-bound)""","""Paulina""","""Paulina (Brown Line)""",41310,true,false,false,false,true,false,false,false,false,false,"""(41.943623, -87.670907)"""
30162,"""W""","""18th (54th/Cermak-bound)""","""18th""","""18th (Pink Line)""",40830,true,false,false,false,false,false,false,false,true,false,"""(41.857908, -87.669147)"""
30374,"""S""","""Clark/Lake (Forest Pk-bound)""","""Clark/Lake""","""Clark/Lake (Blue, Brown, Green…",40380,true,false,true,false,false,false,false,false,false,false,"""(41.885737, -87.630886)"""
30248,"""S""","""Jefferson Park (Forest Pk-boun…","""Jefferson Park""","""Jefferson Park (Blue Line)""",41280,true,false,true,false,false,false,false,false,false,false,"""(41.970634, -87.760892)"""


In [42]:
# Stop information dataset is messy; needs several cleaning operations:
# * convert all columns to snakecase
# * remove location column 
# * rename columns: {'g': 'green', 'brn': 'brown', 'p': 'purple', 'pexp': 'purple_express',
# 'y': 'yellow', 'pnk': 'pink', 'o': 'orange'}
# * rename columns: add _indicator to all boolean variables
line_rename_map = {
    'red': 'red',
    'blue': 'blue',
    'g': 'green', 
    'brn': 'brown', 
    'p': 'purple', 
    'pexp': 'purple_express',
    'y': 'yellow', 
    'pnk': 'pink', 
    'o': 'orange'
}

new_line_col_names = list(line_rename_map.values())

system_info_df = system_info_df.rename(
    {col: utils.to_snake_case(col) for col in system_info_df.columns}
).select(
    pl.exclude('location')
).rename(
    line_rename_map
).with_columns(
    pl.col(new_line_col_names).cast(pl.Int8)
).with_columns(
    pl.col('ada').cast(pl.Int8)
).rename(
    {col: col + '_indicator' for col in new_line_col_names}
).rename(
    {'ada': 'ada_indicator'}
)

In [43]:
system_info_df.head(10)

stop_id,direction_id,stop_name,station_name,station_descriptive_name,map_id,ada_indicator,red_indicator,blue_indicator,green_indicator,brown_indicator,purple_indicator,purple_express_indicator,yellow_indicator,pink_indicator,orange_indicator
i64,str,str,str,str,i64,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
30082,"""E""","""Cicero (Loop-bound)""","""Cicero""","""Cicero (Pink Line)""",40420,1,0,0,0,0,0,0,0,1,0
30151,"""E""","""Central Park (Loop-bound)""","""Central Park""","""Central Park (Pink Line)""",40780,1,0,0,0,0,0,0,0,1,0
30184,"""W""","""Halsted/63rd (Ashland-bound)""","""Halsted""","""Halsted (Green Line)""",40940,1,0,0,1,0,0,0,0,0,0
30044,"""N""","""Cumberland (O'Hare-bound)""","""Cumberland""","""Cumberland (Blue Line)""",40230,1,0,1,0,0,0,0,0,0,0
30092,"""E""","""Racine (O'Hare-bound)""","""Racine""","""Racine (Blue Line)""",40470,0,0,1,0,0,0,0,0,0,0
30253,"""N""","""Paulina (Kimball-bound)""","""Paulina""","""Paulina (Brown Line)""",41310,1,0,0,0,1,0,0,0,0,0
30162,"""W""","""18th (54th/Cermak-bound)""","""18th""","""18th (Pink Line)""",40830,1,0,0,0,0,0,0,0,1,0
30374,"""S""","""Clark/Lake (Forest Pk-bound)""","""Clark/Lake""","""Clark/Lake (Blue, Brown, Green…",40380,1,0,1,0,0,0,0,0,0,0
30248,"""S""","""Jefferson Park (Forest Pk-boun…","""Jefferson Park""","""Jefferson Park (Blue Line)""",41280,1,0,1,0,0,0,0,0,0,0
